## Importamos librerias

In [1]:
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.calibration import LabelEncoder
from keras.layers import LSTM, Dense, Dropout

## Preparativos antes de la RNN

In [2]:
def preprocess_audio(file_path,max_len):
    # Cargar el audio
    signal, sr = librosa.load(file_path,sr=96000)
    # Realizar preénfasis
    #filter_audio = librosa.effects.preemphasis(signal)
    # Extraer MFCCs
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)

    # Realizar padding o recorte
    if mfcc.shape[1] < max_len:
        num_zeros = max_len - mfcc.shape[1]
        padded_mfcc = np.pad(mfcc, ((0, 0), (0, num_zeros)), mode='constant', constant_values=0)
        return padded_mfcc
    else:
        mfcc = mfcc[:, :max_len]
        return mfcc

In [3]:
DATA_DIR = 'Data'
LABELS = ['happy','cat', 'bed']

mfccs = []
labels = []

padding = 500

for label in LABELS:
    path_file = DATA_DIR + f'/{label}'
    for file in os.listdir(path_file):
        file_path = path_file + f'/{file}'
        
        mfcc = preprocess_audio(file_path,padding)
        
        mfccs.append(mfcc)
        labels.append(label)

In [5]:
#Convertimos las listas a numpy array
mfccs_array = np.array(mfccs) #Matriz 3d (5187, 13, max_len + 500)

In [6]:
# Convertir etiquetas a números
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

print("Codificación realizada:")
for idx, label in enumerate(le.classes_):
    print(f"{label} -> {idx}")

Codificación realizada:
bed -> 0
cat -> 1
happy -> 2


In [7]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(mfccs_array, labels_encoded, test_size=0.2, random_state=42)

# Crear modelo LSTM (Long short-term memory)
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 3 clases: 'happy', 'cat', 'bed'

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Entrenamiento

In [8]:
# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

Epoch 1/15
130/130 [==============================] - 17s 93ms/step - loss: 0.9306 - accuracy: 0.5495 - val_loss: 0.7514 - val_accuracy: 0.6802
Epoch 2/15
130/130 [==============================] - 9s 70ms/step - loss: 0.7010 - accuracy: 0.6992 - val_loss: 0.6556 - val_accuracy: 0.7206
Epoch 3/15
130/130 [==============================] - 9s 73ms/step - loss: 0.5743 - accuracy: 0.7628 - val_loss: 0.5963 - val_accuracy: 0.7611
Epoch 4/15
130/130 [==============================] - 11s 82ms/step - loss: 0.5204 - accuracy: 0.7896 - val_loss: 0.5422 - val_accuracy: 0.7890
Epoch 5/15
130/130 [==============================] - 11s 84ms/step - loss: 0.4752 - accuracy: 0.8079 - val_loss: 0.5072 - val_accuracy: 0.7996
Epoch 6/15
130/130 [==============================] - 10s 78ms/step - loss: 0.4510 - accuracy: 0.8260 - val_loss: 0.4387 - val_accuracy: 0.8372
Epoch 7/15
130/130 [==============================] - 11s 88ms/step - loss: 0.4024 - accuracy: 0.8424 - val_loss: 0.4468 - val_accuracy: 0

## Evaluación

In [9]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

33/33 [==============================] - 1s 36ms/step - loss: 0.3886 - accuracy: 0.8516
Loss: 0.38864654302597046
Accuracy: 0.8516377806663513


## Prueba

In [16]:
file_path = 'audio_happy.wav'
preprocessed_audio = preprocess_audio(file_path,padding)
preprocessed_audio = preprocessed_audio.reshape(1, preprocessed_audio.shape[0], preprocessed_audio.shape[1])  # Convertir a formato (1, features, tiempo)
prediction = model.predict(preprocessed_audio)

print(f"{le.inverse_transform([0])[0]} : {prediction[0][0] * 100}%")
print(f"{le.inverse_transform([1])[0]} : {prediction[0][1] * 100}%")
print(f"{le.inverse_transform([2])[0]} : {prediction[0][2] * 100}%")

predicted_label_encoded = np.argmax(prediction, axis=1)[0]
predicted_label = le.inverse_transform([predicted_label_encoded])[0]
print(f"\nLa palabra predicha es: {predicted_label}")

1/1 [==============================] - 0s 32ms/step
bed : 22.82307893037796%
cat : 2.5985581800341606%
happy : 74.57835674285889%

La palabra predicha es: happy
